In [36]:
import os
import sys
import json
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt

sys.path.append('../src')
from settings import YOUTUBE_DIR, TIMESTAMPS_DIR
from youtube import (
    get_nearest_matches, get_video_metadata, generate_team_pairs, generate_team_tournaments,
    search_team_tournament_pairs
)
from search import search, search_top1, get_text_embeddings, get_distilbert_hidden_state

In [12]:
url = 'https://www.youtube.com/watch?v=cXA5Hw2boLA&ab_channel=DotADigest'
video_id, metadata = get_video_metadata(url)
upload_date = metadata['upload_date']
upload_date = datetime.strptime(upload_date, '%Y%m%d')
matches = get_nearest_matches(upload_date)
print(len(matches))

2022-08-18 16:06:01.558 | DEBUG    | utils:query_opendota:28 - 
    SELECT
        match_id,
        start_time,
        matches.leagueid,
        leagues.name as league,
        radiant_team_id,
        radiant_team.name as radiant_name,
        radiant_team.tag as radiant_tag,
        dire_team_id,
        dire_team.name as dire_name,
        dire_team.tag as dire_tag
    FROM
        matches
        join teams as dire_team on matches.dire_team_id = dire_team.team_id
        join teams as radiant_team on matches.radiant_team_id = radiant_team.team_id
        join leagues on matches.leagueid = leagues.leagueid
    WHERE
        start_time >= extract(epoch from timestamp '07-22-2022')
        and start_time < extract(epoch from timestamp '07-25-2022')
    ORDER BY
        start_time desc
    LIMIT
        500
    


288


In [ ]:
title = "SECRET vs TEAM SPIRIT - RAMPAGE! SEMI FINAL - RIYADH MASTERS 2022 Dota 2 Highlights"

In [17]:
hidden_states = get_distilbert_hidden_state([
    'SECRET vs TEAM SPIRIT - RAMPAGE! SEMI FINAL - RIYADH MASTERS 2022 Dota 2 Highlights',
])
hidden_states.shape

torch.Size([1, 12, 768])

In [18]:
hidden_states = get_distilbert_hidden_state([
    'SECRET vs TEAM SPIRIT - RAMPAGE! SEMI FINAL - RIYADH MASTERS 2022 Dota 2 Highlights',
    'SECRET vs SPIRIT - RIYADH MASTERS 2022',
])
hidden_states.shape

torch.Size([2, 22, 768])

In [20]:
embedding = get_text_embeddings(title)

In [25]:
search(
    'SECRET vs TEAM SPIRIT - RAMPAGE! SEMI FINAL - RIYADH MASTERS 2022 Dota 2 Highlights',
    [
        'NAVI vs Aliance - The International 2013',
        'LGD Pushat',
        'SECRET vs SPIRIT - RIYADH MASTERS 2022',  
        'Net drug, ya ne opravdidaus',
    ],
    top=4
)

[(2, 'SECRET vs SPIRIT - RIYADH MASTERS 2022'),
 (0, 'NAVI vs Aliance - The International 2013'),
 (1, 'LGD Pushat'),
 (3, 'Net drug, ya ne opravdidaus')]

In [ ]:
{
    "match_id": 6676488286,
    "start_time": 1658692615,
    "leagueid": 14391,
    "league": "Riyadh Masters by Gamers8",
    "radiant_team_id": 7119388,
    "radiant_name": "Team Spirit",
    "radiant_tag": "TSpirit",
    "dire_team_id": 15,
    "dire_name": "PSG.LGD",
    "dire_tag": "PSG.LGD"
}

In [28]:
generate_team_pairs([{
    "match_id": 6676488286,
    "start_time": 1658692615,
    "leagueid": 14391,
    "league": "Riyadh Masters by Gamers8",
    "radiant_team_id": 7119388,
    "radiant_name": "Team Spirit",
    "radiant_tag": "TSpirit",
    "dire_team_id": 15,
    "dire_name": "PSG.LGD",
    "dire_tag": "PSG.LGD"
}])

{'Team Spirit vs PSG.LGD': 6676488286,
 'PSG.LGD vs Team Spirit': 6676488286,
 'TSpirit vs PSG.LGD': 6676488286,
 'PSG.LGD vs TSpirit': 6676488286}

In [34]:
print(json.dumps({'Team Spirit vs PSG.LGD | Riyadh Masters by Gamers8': 6676488286,
 'PSG.LGD vs Team Spirit | Riyadh Masters by Gamers8': 6676488286,
 'TSpirit vs PSG.LGD | Riyadh Masters by Gamers8': 6676488286,
 'PSG.LGD vs TSpirit | Riyadh Masters by Gamers8': 6676488286}, indent=4))

{
    "Team Spirit vs PSG.LGD | Riyadh Masters by Gamers8": 6676488286,
    "PSG.LGD vs Team Spirit | Riyadh Masters by Gamers8": 6676488286,
    "TSpirit vs PSG.LGD | Riyadh Masters by Gamers8": 6676488286,
    "PSG.LGD vs TSpirit | Riyadh Masters by Gamers8": 6676488286
}


In [33]:
generate_team_tournaments([{
    "match_id": 6676488286,
    "start_time": 1658692615,
    "leagueid": 14391,
    "league": "Riyadh Masters by Gamers8",
    "radiant_team_id": 7119388,
    "radiant_name": "Team Spirit",
    "radiant_tag": "TSpirit",
    "dire_team_id": 15,
    "dire_name": "PSG.LGD",
    "dire_tag": "PSG.LGD"
}])

{'Team Spirit vs PSG.LGD | Riyadh Masters by Gamers8': 6676488286,
 'PSG.LGD vs Team Spirit | Riyadh Masters by Gamers8': 6676488286,
 'TSpirit vs PSG.LGD | Riyadh Masters by Gamers8': 6676488286,
 'PSG.LGD vs TSpirit | Riyadh Masters by Gamers8': 6676488286}

In [35]:
title = 'SECRET vs TEAM SPIRIT - RAMPAGE! SEMI FINAL - RIYADH MASTERS 2022 Dota 2 Highlights'
teams_pair, _, tournament = [t.strip() for t in title.split('-')]
tournament = tournament.replace('Dota 2 Highlights', '')
title = f'{teams_pair} | {tournament}'
print(title)

SECRET vs TEAM SPIRIT | RIYADH MASTERS 2022 


In [38]:
match_id = search_team_tournament_pairs(teams_pair, tournament, matches)
[m for m in matches if m['match_id'] == 6673204572]

6673204572

In [41]:
print(json.dumps([{'match_id': 6673204572,
  'start_time': 1658513208,
  'leagueid': 14391,
  'league': 'Riyadh Masters by Gamers8',
  'radiant_team_id': 1838315,
  'radiant_name': 'Team Secret',
  'radiant_tag': 'Secret',
  'dire_team_id': 7119388,
  'dire_name': 'Team Spirit',
  'dire_tag': 'TSpirit'}], indent=4))

[
    {
        "match_id": 6673204572,
        "start_time": 1658513208,
        "leagueid": 14391,
        "league": "Riyadh Masters by Gamers8",
        "radiant_team_id": 1838315,
        "radiant_name": "Team Secret",
        "radiant_tag": "Secret",
        "dire_team_id": 7119388,
        "dire_name": "Team Spirit",
        "dire_tag": "TSpirit"
    }
]
